Iteration can be done 

In [1]:
import octako.iterator as it
import octako as oc

For creating an iterator, first, it is necessary to create an accessor which accesses the data. Then `MetaAccessors` can be used which perform operations to change the way in which iteration is done.
 
 In this example an iterator will be created in which iterates over a small data set randomly.

In [2]:
data = [0, 1, 2, 3]
to_iterator = oc.in_ >> it.Accessor(data) >> it.Shuffle() >> it.ToIter()

iterator = to_iterator()
print('First: ', iterator.get())
iterator.adv()
print('Second: ', iterator.get())
iterator.adv()
print('Third: ', iterator.get())
iterator.adv()
print('Fourth: ', iterator.get())
iterator.adv()
print('Fifth: ', iterator.get())



First:  0
Second:  1
Third:  2
Fourth:  3
Fifth:  None


In this example an iterator which extends the previous will be created in which iterates over a small data set randomly in batches.

In [3]:
from octako import to_neuron as n
import numpy as np
data = np.array([0, 1, 2, 3])

# create the base pattern for the iterator
BaseToIterator = oc.Stem(oc.nil_ >> it.Accessor(data) >> oc.arg.meta >> it.ToIter())

# Create an iterator with  "MetaAccessors"
# An arm is passed in in order to do the concatenation here
to_iterator = BaseToIterator(meta=oc.Arm(n(it.Shuffle()) >> it.Batch(size=2)))

iterator = to_iterator()
print('First: ', iterator.get())
iterator.adv()
print('Second: ', iterator.get())
iterator.adv()
print('Third: ', iterator.get())


First:  [3 1]
Second:  [2 0]
Third:  None


Here is an example with a reversal on the iterator. Since the present Reverse requires an index be passed in and Batch will pass a 'slice', it is necessary to put Reverse after 'Batch'.

In [4]:
from octako import to_neuron as n
import numpy as np
data = np.array([0, 1, 2, 3])

# create the base pattern for the iterator
BaseToIterator = oc.Stem(oc.nil_ >> it.Accessor(data) >> oc.arg.meta >> it.ToIter())

# Create an iterator with  "MetaAccessors"
# An arm is passed in in order to do the concatenation here
to_iterator = BaseToIterator(meta=oc.Arm(oc.in_>> it.Batch(size=2)  >> it.Reverse()))

iterator = to_iterator()
print('First: ', iterator.get())
iterator.adv()
print('Second: ', iterator.get())
iterator.adv()
print('Third: ', iterator.get())

First:  [2 3]
Second:  [0 1]
Third:  None


Since iterators go hand in hand with looping, the example below shows a loop (Repeat) used with an iterator. Repeat does not do any extra processing on the output of the iterator in this case.

In [5]:
from octako import to_neuron as n
from octako import flow
import numpy as np
data = np.array([0, 1, 2, 3])

# create the base pattern for the iterator
BaseIterator = oc.Stem(oc.nil_ >> it.Accessor(data) >> oc.arg.meta >> it.ToIter())

# Create an iterator with  "MetaAccessors"
# An arm is passed in in order to do the concatenation here

loop = BaseToIterator(meta=oc.Arm(oc.in_>> it.Reverse())) >>flow.Repeat(
    it.Iterate(),
    output_all=True
)

print('Output: ', loop())

Output:  [3, 2, 1, 0, None]


In this case, we will add some processing to the Repeat control structure.

In [6]:
from octako import to_neuron as n
from octako import flow
import numpy as np
data = np.array([0, 1, 2, 3])

# create the base pattern for the iterator
BaseIterator = oc.Stem(oc.nil_ >> it.Accessor(data) >> oc.arg.meta >> it.ToIter())

# Create an iterator with  "MetaAccessors"
# An arm is passed in in order to do the concatenation here

loop = BaseToIterator(meta=oc.Arm(oc.in_>> it.Reverse())) >>flow.Repeat(
    it.Iterate() >> flow.Gate(neuron=lambda x: x + 2),
    output_all=True
)

print('Output: ', loop())

Output:  [5, 4, 3, 2, None]


Since the Iterator and Repeat with the iteration gated is a common pattern, this pattern can be combined into one stem for reuse.

In [7]:
from octako import to_neuron as n
from octako import flow
import numpy as np
data = np.array([0, 1, 2, 3])

# create the base pattern for the iterator
BaseIterator = oc.Stem(oc.in_ >> it.Accessor() >> oc.arg.meta >> it.ToIter())

IterateOn = oc.Stem(BaseIterator(meta=n(oc.arg.meta)) >>flow.Repeat(
    it.Iterate() >> flow.Gate(neuron=oc.arg.process),
    output_all=True
))

iterate_on = IterateOn(
    meta=n(oc.in_ >> it.Batch(size=2)), 
    process=n(lambda x: x + 2)
)

print(iterate_on(data))

[array([2, 3]), array([4, 5]), None]
